In [ ]:
import json
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split


In [ ]:
# Load training and test data
with open('training.json') as f:
    training_data = json.load(f)

with open('test.json') as f:
    test_data = json.load(f)

In [ ]:
# Extract features and labels
def extract_features_and_labels(data):
    X = [[item['sepal_length'], item['sepal_width'], item['petal_length'], item['petal_width']] for item in data]
    y = [item['species'] for item in data]
    return np.array(X), np.array(y)

X_train_raw, y_train_raw = extract_features_and_labels(training_data)
X_test_raw, y_test_raw = extract_features_and_labels(test_data)

In [ ]:
# Preprocess features and labels
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train_raw)
y_test = lb.transform(y_test_raw)

In [ ]:
# MLP Model - Using the same structure as the original code with backpropagation
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        np.random.seed(1)  # For reproducibility
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.learning_rate = learning_rate
        self.bias_hidden = np.random.rand(1, hidden_size)
        self.bias_output = np.random.rand(1, output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        # Forward pass
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)
        return self.final_output

    def backward(self, X, y, output):
        # Backpropagation
        error = y - output
        d_output = error * self.sigmoid_derivative(output)

        error_hidden = d_output.dot(self.weights_hidden_output.T)
        d_hidden = error_hidden * self.sigmoid_derivative(self.hidden_output)

        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(d_output) * self.learning_rate
        self.weights_input_hidden += X.T.dot(d_hidden) * self.learning_rate
        self.bias_output += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
        self.bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs=10000):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        return self.forward(X)


In [ ]:
# Initialize the MLP
mlp = MLP(input_size=4, hidden_size=5, output_size=3)

# Train the MLP
mlp.train(X_train, y_train, epochs=10000)

# Make predictions on test data
predictions = mlp.predict(X_test)

# Convert predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(y_test, axis=1)
class_labels = lb.classes_

# Print predictions and actual labels
for i in range(len(X_test)):
    print(f"Prediction: {class_labels[predicted_labels[i]]}, Actual: {class_labels[actual_labels[i]]}")

Prediction: setosa, Actual: setosa
Prediction: virginica, Actual: virginica
Prediction: versicolor, Actual: versicolor
